In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

import numpy as np
import arrayfire as af
from matplotlib import pyplot as plt

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import wave_equation_2d as w2d
from dg_maxwell import isoparam
from dg_maxwell import lagrange
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [3]:
params.mesh_file = '../read_and_plot_mesh/mesh/particle_in_rectangle.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)

/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


gmshTranslator: Initializing...
gmshTranslator: Mesh has 988 nodes.
gmshTranslator: Mesh has 374 elements.
gmshTranslator: Processed 1366 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


# Vectorized Surface term developmental Stage

In [32]:
# 1. Find L_p(1) and L_p(-1)
Lp = advec_var.lagrange_coeffs
Lq = advec_var.lagrange_coeffs

Lp_1           = utils.polyval_1d(Lp, af.constant(1, d0 = 1, dtype = af.Dtype.f64))
Lp_1_slow_tile = af.moddims(af.transpose(af.tile(Lp_1, d0 = 1, d1 = params.N_LGL,
                                                 d2 = params.N_LGL)),
                            d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

Lp_minus_1           = utils.polyval_1d(Lp, af.constant(-1, d0 = 1, dtype = af.Dtype.f64))
Lp_minus_1_slow_tile = af.moddims(af.transpose(af.tile(Lp_minus_1, d0 = 1, d1 = params.N_LGL,
                                                       d2 = params.N_LGL)),
                                  d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)

Lq_1 = Lp_1.copy()
Lq_1_quick_tile = af.tile(Lq_1, d0 = params.N_LGL, d1 = params.N_LGL)

Lq_minus_1 = Lp_minus_1.copy()
Lq_minus_1_quick_tile = af.tile(Lq_minus_1, d0 = params.N_LGL, d1 = params.N_LGL)
# [LOOKS FINE]


In [33]:
# 2. Find Lp(xi) and Lq(eta) [N_LGL * N_LGL, N_LGL]

Lp_xi_i = utils.polyval_1d(Lp, advec_var.xi_LGL)

# print(Lp_xi_i)

Lq_eta_j = utils.polyval_1d(Lq, advec_var.eta_LGL)
# print(Lq_eta_j)

arrayfire.Array()
Type: double

[8 8 1 1]
    1.0000     0.0000     0.0000     0.0000     0.0000     0.0000    -0.0000     0.0000 
    0.0000     1.0000     0.0000     0.0000    -0.0000    -0.0000    -0.0000     0.0000 
   -0.0000    -0.0000     1.0000     0.0000    -0.0000    -0.0000     0.0000     0.0000 
    0.0000     0.0000     0.0000     1.0000    -0.0000    -0.0000    -0.0000    -0.0000 
   -0.0000    -0.0000    -0.0000    -0.0000     1.0000     0.0000     0.0000     0.0000 
   -0.0000    -0.0000    -0.0000    -0.0000     0.0000     1.0000    -0.0000     0.0000 
   -0.0000    -0.0000    -0.0000    -0.0000    -0.0000    -0.0000     1.0000    -0.0000 
    0.0000     0.0000     0.0000     0.0000     0.0000    -0.0000    -0.0000     1.0000 




In [37]:
# Different modes of tiling Lp

# Quick tiling

foo = af.range(params.N_LGL)

foo_quick_tile = af.tile(foo, d0 = params.N_LGL, d1 = params.N_LGL)
# print(foo_quick_tile)

foo_slow_tile = af.moddims(af.transpose(af.tile(foo, d0 = 1, d1 = params.N_LGL, d2 = params.N_LGL)),
                           d0 = params.N_LGL * params.N_LGL, d1 = params.N_LGL)
# print(foo_slow_tile)

In [42]:
# Quick/slow tiling for foo_1

foo_1 = foo_quick_tile[:params.N_LGL]

# Quick tile
foo_1_quick_tile = af.tile(foo_1, d0 = params.N_LGL)

foo_1_slow_tile = af.tile(af.reorder(foo_1, d0 = 2, d1 = 0, d2 = 1), d1 = )
print(foo_1_slow_tile)

arrayfire.Array()
Type: float

[1 8 8 1]
    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 

    0.0000     1.0000     2.0000     3.0000     4.0000     5.0000     6.0000     7.0000 



